In [3]:
import requests
import pandas as pd
import io
import json
import re
from pandas_datareader import data, wb
import numpy as np

In [5]:
#Campeonato Quantamental
#Pegando dados P/L



headers = {
    'authority': 'www.oceans14.com.br',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Mobile Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    # coloque aqui a url de qualquer canto que seja do site oceans14
    'referer': 'https://www.oceans14.com.br/acoes/itau-unibanco/itub/balanco-dividendos',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7,la;q=0.6',
    'cookie': 'ASP.NET_SessionId=jypd15qvl1yu3ypbgqzecxjj; uid=0EAB27F50DB06E1775959795C4949E2D; origem=https://l.facebook.com/; contadorViewsAcoes=3',
}

params = (
    ######IDENTIFICAR TICKER DA ACAO######
    ('papel', 'ITUB3'),
    ('periodo', '10a'),
)

response = requests.get('https://www.oceans14.com.br/rendaVariavel/respostaAjax/gHistoricoPl.aspx', headers=headers, params=params)
dframe = pd.json_normalize(response.json())
jason = response.text
normal = json.loads(jason)
datas = []
valores = []
for row in normal["rows"]:
    date = row['c'][0]['v']
    formatedDate = re.search(r"(?<=Date\()[^\):]+(?=:?\))", date).group(0)
    simpleData = formatedDate.replace(',', '-').replace(' ', '')
    separatedData = simpleData.split('-')
    correctDataList = ''
    for number in separatedData:
        if number != '0':
            correctDataList += ("%02d" % int(number)) + "-"
    correctDataList = correctDataList[:-1]
    datas.append(correctDataList)
    valores.append(row['c'][1]['v'])
frameDict = {
    "Date": datas,
    "valores": valores
}

In [8]:
dframe = pd.DataFrame(data=frameDict)
df = pd.DataFrame()
df['valores'] = dframe['valores'].values.tolist()
valores = df['valores']
dframe

,Date,valores
0,2010-06-13,11.07
1,2010-06-14,11.11
2,2010-06-15,11.00
3,2010-06-16,10.72
4,2010-06-19,10.87
...,...,...
2466,2020-06-03,10.34
2467,2020-06-06,10.68
2468,2020-06-07,10.43
2469,2020-06-08,10.59


In [ ]:
dframe.set_index('Date', inplace=True)

In [11]:
dframe

,valores
Date,
2010-06-13,11.07
2010-06-14,11.11
2010-06-15,11.00
2010-06-16,10.72
2010-06-19,10.87
...,...
2020-06-03,10.34
2020-06-06,10.68
2020-06-07,10.43


In [12]:
#url ='https://fundamentei.com/br/petr'
#df = pd.read_html(url)

start_date = '2010-06-11'
end_date = '2020-06-08'
df = data.DataReader('ITUB3.SA', 'yahoo', start_date, end_date) #digitar ticker
df['Preco_Lucro'] = dframe['valores']
tabela = df[['Close', 'Preco_Lucro']]
Mylist = tabela.columns.values.tolist()
dfim = pd.concat([df, dframe], axis=1)

ValueError: ignored